In [1]:
!pip install gudhi==3.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import gudhi
import numpy as np

In [3]:
# Datos originales

data = {
    0: [3,3,2,2,3,1,1,2,2,2,2,4,4.7,4,4,4.7,4.5],
    1: [4,4,5,4,4,3,4,3,4,5,4,4,4.8,2,4.3,4.4,4.9],
    2: [2,2,1,1,2,1,2,3,2,2,1,2,4.8,2,3.6,4.5,4.8],
    3: [2,1,1,2,3,3,3,2,3,2,2,3,4.9,3,4.1,4.9,4.7],
    4: [2,2,2,3,3,2,2,3,3,2,3,2,4.3,3,3.0,3.0,4.2],
    5: [5,3,3,3,5,4,4,3,5,2,4,2,5.0,1,4.8,4.7,None],
    6: [2,1,3,2,2,2,3,2,2,2,2,1,4.4,2,4.1,4.1,3.8],
    7: [5,3,5,3,4,5,4,3,2,5,3,2,4.3,2,4.9,4.4,4.2],
    8: [2,2,1,4,2,5,2,2,3,1,3,3,4.6,2,4.4,None,3.9],
    9: [3,2,2,3,4,3,4,3,2,2,2,3,4.0,3,4.7,3.2,3.0],
    10: [1,2,2,3,1,3,2,1,2,2,3,3,4.7,2,4.4,4,4.2],
    11: [4,1,2,1,1,1,1,1,2,2,2,1,3.7,4,3.8,4.4,3.7],
    12: [3,4,2,3,4,1,2,3,2,2,4,3,4.3,3,4.7,4.8,4.2],
    13: [1,2,1,1,2,1,3,3,1,1,1,1,3.2,1,4,3.5,3.7],
    14: [1,2,2,2,2,1,1,1,2,2,2,3,4.5,2,4.0,3.4,4.1],
    15: [2,1,2,4,3,1,2,2,2,2,3,2,4.7,3,3.3,4.3,4.1]
}


# Rellenar valores faltantes
def fill_missing_with_mean(data):
    names = list(data.keys())
    matrix = np.array(list(data.values()), dtype=float)
    col_means = np.nanmean(np.where(matrix == None, np.nan, matrix), axis=0)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i, j] is None or np.isnan(matrix[i, j]):
                matrix[i, j] = col_means[j]
    
    filled_data = {names[i]: matrix[i].tolist() for i in range(len(names))}
    
    return filled_data

# Corrección de notas
data_filled = fill_missing_with_mean(data)
for datas in data_filled:
    for datos in [-1, -2, -3, -5]:
          data_filled[datas][datos] = 5-data_filled[datas][datos]
        

# Estandarización de los datos
def transponer_matriz(A):
    return [list(fila) for fila in zip(*A)]
def estand(A):
    keys = list(A.keys())
    a = [A[k] for k in keys]
    a_t = transponer_matriz(a)
    for columna in a_t:
        mini = min(columna)
        maxi = max(columna)
        if maxi - mini != 0:
            for i in range(len(columna)):
                columna[i] = (columna[i] - mini) / (maxi - mini)
        else:
            for i in range(len(columna)):
                columna[i] = 0
    a_std = transponer_matriz(a_t)
    return a_std

points = estand(data_filled)

In [4]:
# Matriz de distancia triangular
def Distance_Matrix(distance, points):
    matrix = []
    for i in range(len(points)):
        row = []
        for j in range(i):
            row.append(distance(points[i], points[j]))
        matrix.append(row)

    return matrix


# Distancia Euclidea, se puede cambiar a otras
def distance(a, b):
    res = 0
    for i in range(len(a)):
        res += (a[i] - b[i])**2
    return res**0.5

# Máxima distancia para el complejo simplicial
Matrix = Distance_Matrix(distance, points)

distances = []
for row in Matrix:
    for el in row:
        distances.append(el)

distances.sort()

# Selección de epsilon mediante la mediana
n = len(distances)
epsilon = (distances[n//2] + distances[n//2+1])/2

# Esto es para hacer el complejo
rips_complex = gudhi.RipsComplex(
    distance_matrix = Matrix, 
    max_edge_length = epsilon
)

# Esto es una estructura para que le de las caras del complejo
simplex_tree = rips_complex.create_simplex_tree(max_dimension=16)


# Creación del vector f, tal que f_n = # Caras de dimensión n
f_vector = [0 for i in range(simplex_tree.dimension()+1)]
for face in simplex_tree.get_filtration():
    f_vector[len(face[0])-1] += 1

# Estabilidad de la estructura
k = len(points)
stab = 0
for i in range(len(f_vector)):
    stab += f_vector[i] - (k+1)
stab /= (2**(k+1)-(k+2))
print(stab)

0.003067437850046546


In [5]:
# Función que retorna el conjunto Estrella de un conjunto
def star(s, simplex):
    star_set = []
    for face in simplex.get_filtration():
        for el in s:
            if(el not in face[0]):
                break
        else:
            star_set.append(face[0])
    return star_set

# Viabilidad de cada uno de los puntos
for i in range(len(points)):
    via = (len(star([i], simplex_tree)) - 1) / (2**k - 1)
    print(via)

0.0035248340581368735
7.629510948348211e-05
0.002059967956054017
0.0037689784084840162
0.0038910505836575876
7.629510948348211e-05
0.003402761882963302
6.103608758678569e-05
0.0027924010070954452
0.0007324330510414282
0.003402761882963302
0.00022888532845044633
0.0012512397955291067
0.00022888532845044633
0.0031586175326161594
0.0038910505836575876


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0869eb2d-21ed-4b31-a4f6-d09a7c1322ce' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>